## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---
## First, I'll compute the camera calibration using chessboard images

In [1]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pickle
%matplotlib qt

camera_params = pickle.load( open( "camera_calibration.p", "rb" ) )
mtx = camera_params['mtx']
dist = camera_params['dist']
rvecs = camera_params['rvecs']
tvecs = camera_params['tvecs']


In [2]:
def undistort(img, mtx, dist):
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    return undist  

def binary_sobelx(color_img,kernel_size=3,sx_thresh=(20, 100), gauss_blur=False, gauss_k_size = 3):
    gray = cv2.cvtColor(np.copy(color_img), cv2.COLOR_RGB2GRAY)  
    
    if gauss_blur:
        smoothed = gaussian_blur(gray,gauss_k_size)
        sobelx = cv2.Sobel(smoothed, cv2.CV_64F, 1, 0,ksize=kernel_size) # Take the derivative in x
    else:
        sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0,ksize=kernel_size) # Take the derivative in x

    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1
    
    return sxbinary

def sobel_mag_thresh(color_img, kernel_size=3, mag_thresh=(30, 100)):
    
    # 1) Convert to grayscale
    gray = cv2.cvtColor(np.copy(color_img), cv2.COLOR_RGB2GRAY)
    
    # 2) Take both Sobel x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=kernel_size)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=kernel_size)
    
    # 3) Calculate the magnitude
    mag = np.sqrt(sobelx**2 + sobely**2)
    
    # 4) Scale to 8-bit (0 - 255) and convert to type = np.uint8
    scale_factor = np.max(mag)/255 
    norm_mag = (mag/scale_factor).astype(np.uint8)     
    
    # 5) Create a binary mask where mag thresholds are met
    binary_mag = np.zeros_like(norm_mag)
    binary_mag[(norm_mag >= mag_thresh[0]) & (norm_mag <=mag_thresh[1])] = 1
    
    return binary_mag

def binary_s_channel(color_img,s_thresh=(150, 255)):
    
    hls = cv2.cvtColor(np.copy(color_img), cv2.COLOR_RGB2HLS)
    s_channel = hls[:,:,2]
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1    
    
    return s_binary

def binary_r_channel(color_img,r_thresh=(200,255)):
    img = np.copy(color_img)
    #red channel
    r_channel = img[:,:,0]
    r_binary = np.zeros_like(r_channel)
    r_binary[(r_channel >= r_thresh[0]) & (r_channel <= r_thresh[1])] = 1      
    
    return r_binary 

def binary_filters(s_x = True, bin_sx=0,s_mag=True, bin_s_mag=0, s_ch=True,bin_s_ch=0,r_ch=True,bin_r_ch=0):
    
    #init
    if s_x == True:
        binary_mask = np.zeros_like(bin_sx)
    elif s_mag == True:
        binary_mask = np.zeros_like(bin_s_mag)
    elif s_ch == True:
        binary_mask = np.zeros_like(bin_s_ch)
    elif r_ch == True:
        binary_mask = np.zeros_like(bin_r_ch)

    #apply only specific filters
    if (s_x == True) :
        binary_mask[(bin_sx ==1) | (binary_mask ==1)] = 1                
    if (s_mag == True):
        binary_mask[(bin_s_mag ==1) | (binary_mask ==1)] = 1     
    if (s_ch == True):
        binary_mask[(bin_s_ch ==1) | (binary_mask ==1)] = 1
    if (r_ch == True):
        binary_mask[(bin_r_ch ==1) | (binary_mask ==1)] = 1 
 
    return binary_mask

def colors_and_gradients(warped):
    bin_sobelx = binary_sobelx(warped,kernel_size=5,sx_thresh=(20, 100),gauss_blur=True)
    bin_s_channel = binary_s_channel(warped,s_thresh=(150, 255))
    bin_r_channel =  binary_r_channel(warped,r_thresh=(200,255))
    bin_sobel_mag = sobel_mag_thresh(warped, kernel_size=5, mag_thresh=(30, 100)) 

    count_light_in_red_ch =  np.sum(bin_r_channel[:,:])

    if count_light_in_red_ch > 150000:
        binary_or_img = binary_filters(s_x = True, bin_sx = bin_sobelx,
                                   s_mag=True, bin_s_mag = bin_sobel_mag, 
                                   s_ch=True,bin_s_ch=bin_s_channel,
                                   r_ch=False,bin_r_ch=bin_r_channel)
    else:
        binary_or_img = binary_filters(s_x = True, bin_sx = bin_sobelx,
                                   s_mag=True, bin_s_mag = bin_sobel_mag, 
                                   s_ch=True,bin_s_ch=bin_s_channel,
                                   r_ch=True,bin_r_ch=bin_r_channel)
    return binary_or_img,bin_sobelx,bin_s_channel,bin_r_channel,bin_sobel_mag

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)



def corners_unwarp(img, nx, ny, mtx, dist):
    offset = 100
    img_size = (img.shape[1], img.shape[0])
    ret, corners = cv2.findChessboardCorners(img, (nx,ny), None)

    if ret == True: 

        src = np.float32([corners[0], corners[nx-1], corners[-1], corners[-nx]])

        dst = np.float32([[offset, offset], [img_size[0]-offset, offset], 
                          [img_size[0]-offset, img_size[1]-offset], [offset, img_size[1]-offset]])
        
        #warped, M, invM = warper(img, src, dst)
        M = cv2.getPerspectiveTransform(src, dst)
        warped = cv2.warpPerspective(img,M,img_size,flags=cv2.INTER_LINEAR)
        
        return warped
    else:
        return 0,0

def warper(img, src, dst):

    # Compute and apply perpective transform
    img_size = (img.shape[1], img.shape[0])
    M = cv2.getPerspectiveTransform(src, dst)
    #warped = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_NEAREST)  # keep same size as input image
    warped = cv2.warpPerspective(img,M,img_size,flags=cv2.INTER_LINEAR)
    invM = cv2.getPerspectiveTransform(dst,src)
    
    return warped, M, invM
    

def convert_to_gray(img,mpimgImread=True):
    if (mpimgImread==True):
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        #convert to three channels
        gray = cv2.cvtColor(gray, cv2.COLOR_GRAY2RGB)    
        return gray
    else:
        return cv2.cvtColor(img, cv2.COLOR_BRG2GRAY)
    
    
def getWarpSrcPts(img):

    x_offset_up = 550
    x_offset_low = 170
    y_offset_up = 250
    y_car_hood = 30
    #lower left corner of Trapezoid
    P1 = (x_offset_low,img.shape[0]-y_car_hood)
    #upper left corner of Trapezoid
    P2 = (x_offset_up, img.shape[0]-y_offset_up) 
    #upper right corner of Trapezoid
    P3 = (int(round(img.shape[1]-x_offset_up)), img.shape[0]-y_offset_up)
    #lower right corner of Trapezoid
    P4 = (img.shape[1]-x_offset_low,img.shape[0]-y_car_hood)
    
    return  np.float32([P3,P4,P1,P2]) 

def getWarpDstPts(img):

    x_offset = 300
    y_offset_up = 50
    #lower left corner of Trapezoid
    #P1 = (x_offset_low,img.shape[0])
    #upper left corner of Trapezoid
    #P2 = (x_offset_up, y_offset_up) 
    #upper right corner of Trapezoid
    #P3 = (img.shape[1]-x_offset_up-10, y_offset_up)
    #lower right corner of Trapezoid
    #P4 = (img.shape[1]-x_offset_low,img.shape[0])

    #lower left corner of Trapezoid
    P1 = (x_offset,img.shape[0])
    #upper left corner of Trapezoid
    P2 = (x_offset, y_offset_up) 
    #upper right corner of Trapezoid
    P3 = (img.shape[1]-x_offset, y_offset_up)
    #lower right corner of Trapezoid
    P4 = (img.shape[1]-x_offset,img.shape[0])    
    
    
    return  np.float32([P3,P4,P1,P2])

def merge_color_channels(img):

    channel1 = img[:,:,0]
    channel2 = img[:,:,1]
    channel3 = img[:,:,2]
    
    binary_img = np.zeros_like(channel1)
    binary_img[(channel1 > 0) | (channel2 > 0) | (channel3 > 0) ]=1

    return binary_img

def undistort_and_warp(img):
    undist_img = undistort(img, mtx, dist)
    
    src = getWarpSrcPts(undist_img) 
    dst = getWarpDstPts(undist_img) 

    warped,M,invM = warper(undist_img, src, dst)    
    

    return warped,M,invM

## Run the calibration process only once

In [3]:
#mtx, dist, rvecs, tvecs  = run_calibration_process()

## Testing: undistort and warp chessboard images as test

In [4]:
filename = 'camera_cal/calibration3.jpg'
img1 = mpimg.imread(filename)

imNum1 = filename.split('camera_cal/calibration')[1].split('.jpg')[0]

nx = 9
ny = 6

undistorted1= undistort(img1, mtx, dist)
gray1 = convert_to_gray(undistorted1)
top_down1 = corners_unwarp(gray1, nx, ny, mtx, dist)

#open a second image with different nx and ny
filename = 'camera_cal/calibration1.jpg'
img2 = mpimg.imread(filename)

imNum2 = filename.split('camera_cal/calibration')[1].split('.jpg')[0]
       
nx = 9
ny = 5

undistorted2 = undistort(img2, mtx, dist)
gray2 = convert_to_gray(undistorted2)
top_down2 = corners_unwarp(gray2, nx, ny, mtx, dist)


f, ((ax1, ax2), (ax3,ax4),(ax5,ax6)) = plt.subplots(3,2, figsize=(24, 9))
f.tight_layout()
ax1.imshow(img1)
ax1.set_title('Original calibration image {}'.format(imNum1), fontsize=10)
ax3.imshow(undistorted1)
ax3.set_title('Undistorted image {}'.format(imNum1), fontsize=10)
ax5.imshow(top_down1)
ax5.set_title('Undistorted and Warped  image {}'.format(imNum1), fontsize=10)

ax2.imshow(img2)
ax2.set_title('Original calibration image {}'.format(imNum2), fontsize=10)
ax4.imshow(undistorted2)
ax4.set_title('Undistorted image {}'.format(imNum2), fontsize=10)
ax6.imshow(top_down2)
ax6.set_title('Undistorted and Warped image {}'.format(imNum2), fontsize=10)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.05)

f.savefig('output_images/undistort_warp_chessboard_examples.jpg')

## Testing: undistort both road test images for validation



In [5]:
#undistorting straight line images
test_img_1 = 'test_images/straight_lines1.jpg'
test_img_2 = 'test_images/straight_lines2.jpg'
#test_img_1 = 'camera_cal/calibration1.jpg' 
img1 = plt.imread(test_img_1)
img2 = plt.imread(test_img_2)
test_img_1_undistorted = undistort(img1, mtx, dist)
test_img_2_undistorted = undistort(img2, mtx, dist)


f, ((ax1,ax2),(ax3,ax4)) = plt.subplots(2, 2, figsize=(18, 7))
f.tight_layout()
ax1.imshow(img1)
ax1.set_title('Original Image 1', fontsize=10)
ax2.imshow(test_img_1_undistorted)
ax2.set_title('Undistorted Image 1', fontsize=10)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

ax3.imshow(img2)
ax3.set_title('Original Image 2', fontsize=10)
ax4.imshow(test_img_2_undistorted)
ax4.set_title('Undistorted Image 2', fontsize=10)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

plt.imsave('output_images/straight_lines1_undist.jpg',test_img_1_undistorted )
plt.imsave('output_images/straight_lines2_undist.jpg',test_img_2_undistorted )
# Save the full figure...
f.savefig('output_images/straight_lines_undist.jpg')

## Trying different HLS and Sobel thresholds


In [6]:
'''
filename1 = 'output_images/straight_lines1_undist.jpg'
image1 = plt.imread(filename1)
result11,result12BW,tmp = saturSobelRed(image1, s_thresh=(120, 255), sx_thresh=(10, 100),gauss_blur=True,gauss_k_size=3)
result12,result12BW,tmp = saturSobelRed(image1, s_thresh=(150, 255), sx_thresh=(20, 100),gauss_blur=True,gauss_k_size=3)
result13,result13BW,tmp = saturSobelRed(image1, s_thresh=(255, 255), sx_thresh=(40, 100),gauss_blur=True,gauss_k_size=3)

filename2 = 'output_images/straight_lines2_undist.jpg'
image2 = plt.imread(filename2)
result21,result21BW,tmp = saturSobelRed(image2, s_thresh=(120, 255), sx_thresh=(10, 100),gauss_blur=True,gauss_k_size=3)
result22,result22BW ,tmp= saturSobelRed(image2, s_thresh=(150, 255), sx_thresh=(20, 100),gauss_blur=True,gauss_k_size=3)
result23,result23BW ,tmp= saturSobelRed(image2, s_thresh=(255, 255), sx_thresh=(40, 100),gauss_blur=True,gauss_k_size=3)

# Plot the result
f, ((ax1,ax2),(ax3,ax4),(ax5,ax6)) = plt.subplots(3,2, figsize=(18, 7))
f.tight_layout()
ax1.imshow(result11)
ax1.set_title('Saturation and Sobel params set 1', fontsize=10)
ax3.imshow(result12)
ax3.set_title('Saturation and Sobel params set 2', fontsize=10)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
ax5.imshow(result13)
ax5.set_title('Saturation and Sobel params set 3', fontsize=10)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

ax2.imshow(result21)
ax2.set_title('Saturation and Sobel params set 1', fontsize=10)
ax4.imshow(result22)
ax4.set_title('Saturation and Sobel params set 2', fontsize=10)
ax6.imshow(result23)
ax6.set_title('Saturation and Sobel params set 3', fontsize=10)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
'''

"\nfilename1 = 'output_images/straight_lines1_undist.jpg'\nimage1 = plt.imread(filename1)\nresult11,result12BW,tmp = saturSobelRed(image1, s_thresh=(120, 255), sx_thresh=(10, 100),gauss_blur=True,gauss_k_size=3)\nresult12,result12BW,tmp = saturSobelRed(image1, s_thresh=(150, 255), sx_thresh=(20, 100),gauss_blur=True,gauss_k_size=3)\nresult13,result13BW,tmp = saturSobelRed(image1, s_thresh=(255, 255), sx_thresh=(40, 100),gauss_blur=True,gauss_k_size=3)\n\nfilename2 = 'output_images/straight_lines2_undist.jpg'\nimage2 = plt.imread(filename2)\nresult21,result21BW,tmp = saturSobelRed(image2, s_thresh=(120, 255), sx_thresh=(10, 100),gauss_blur=True,gauss_k_size=3)\nresult22,result22BW ,tmp= saturSobelRed(image2, s_thresh=(150, 255), sx_thresh=(20, 100),gauss_blur=True,gauss_k_size=3)\nresult23,result23BW ,tmp= saturSobelRed(image2, s_thresh=(255, 255), sx_thresh=(40, 100),gauss_blur=True,gauss_k_size=3)\n\n# Plot the result\nf, ((ax1,ax2),(ax3,ax4),(ax5,ax6)) = plt.subplots(3,2, figsize=(18

## Sobel kernels

In [7]:
'''
filename1 = 'output_images/straight_lines1_undist.jpg'
image1 = plt.imread(filename1)
result11,result11BW,tmp = saturSobelRed(image1, s_thresh=(150, 255), sx_thresh=(20, 100),kernelS=3)
result12,result12BW,tmp = saturSobelRed(image1, s_thresh=(150, 255), sx_thresh=(20, 100),kernelS=5)
result13,result13BW,tmp = saturSobelRed(image1, s_thresh=(150, 255), sx_thresh=(20, 100),kernelS=15)

filename2 = 'output_images/straight_lines2_undist.jpg'
image2 = plt.imread(filename2)
result21,result21BW,tmp = saturSobelRed(image2, s_thresh=(150, 255), sx_thresh=(20, 100),kernelS=3)
result22,result22BW ,tmp= saturSobelRed(image2, s_thresh=(150, 255), sx_thresh=(20, 100),kernelS=5)
result23,result23BW,tmp = saturSobelRed(image2, s_thresh=(150, 255), sx_thresh=(20, 100),kernelS=15)

# Plot the result
f, ((ax1,ax2),(ax3,ax4),(ax5,ax6)) = plt.subplots(3,2, figsize=(18, 7))
f.tight_layout()
ax1.imshow(result11)
ax1.set_title('Saturation and Sobel params set 1', fontsize=10)
ax3.imshow(result12)
ax3.set_title('Saturation and Sobel params set 2', fontsize=10)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
ax5.imshow(result13)
ax5.set_title('Saturation and Sobel params set 3', fontsize=10)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

ax2.imshow(result21)
ax2.set_title('Saturation and Sobel params set 1', fontsize=10)
ax4.imshow(result22)
ax4.set_title('Saturation and Sobel params set 2', fontsize=10)
ax6.imshow(result23)
ax6.set_title('Saturation and Sobel params set 3', fontsize=10)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

'''

"\nfilename1 = 'output_images/straight_lines1_undist.jpg'\nimage1 = plt.imread(filename1)\nresult11,result11BW,tmp = saturSobelRed(image1, s_thresh=(150, 255), sx_thresh=(20, 100),kernelS=3)\nresult12,result12BW,tmp = saturSobelRed(image1, s_thresh=(150, 255), sx_thresh=(20, 100),kernelS=5)\nresult13,result13BW,tmp = saturSobelRed(image1, s_thresh=(150, 255), sx_thresh=(20, 100),kernelS=15)\n\nfilename2 = 'output_images/straight_lines2_undist.jpg'\nimage2 = plt.imread(filename2)\nresult21,result21BW,tmp = saturSobelRed(image2, s_thresh=(150, 255), sx_thresh=(20, 100),kernelS=3)\nresult22,result22BW ,tmp= saturSobelRed(image2, s_thresh=(150, 255), sx_thresh=(20, 100),kernelS=5)\nresult23,result23BW,tmp = saturSobelRed(image2, s_thresh=(150, 255), sx_thresh=(20, 100),kernelS=15)\n\n# Plot the result\nf, ((ax1,ax2),(ax3,ax4),(ax5,ax6)) = plt.subplots(3,2, figsize=(18, 7))\nf.tight_layout()\nax1.imshow(result11)\nax1.set_title('Saturation and Sobel params set 1', fontsize=10)\nax3.imshow(r

## Testing perspective transform with color images of roads


In [8]:

filename1 = 'output_images/straight_lines1_undist.jpg'
image1 = plt.imread(filename1)
src = getWarpSrcPts(image1) 
dst = getWarpDstPts(image1) 

warped1,M,invM = warper(image1, src, dst)
plt.imsave('output_images/straight_lines1_warped.jpg',warped1)
#left reference line warped image
cv2.line(warped1, (dst[2][0], dst[2][1]), (dst[3][0], dst[3][1]), [0,255,0], 4)
cv2.circle(warped1, (dst[2][0], dst[2][1]), 2, [0,255,0], 12)
cv2.circle(warped1, (dst[3][0], dst[3][1]), 2, [0,255,0], 12)

#right reference line warped image
cv2.line(warped1, (dst[0][0], dst[0][1]), (dst[1][0], dst[1][1]), [255,0,0], 4)
cv2.circle(warped1, (dst[0][0], dst[0][1]), 2, [255,0,0], 12)
cv2.circle(warped1, (dst[1][0], dst[1][1]), 2, [255,0,0], 12)

#left reference line original image
cv2.line(image1, (src[2][0], src[2][1]), (src[3][0], src[3][1]), [0,255,0], 4) 
cv2.line(image1, (src[2][0], src[2][1]), (src[1][0], src[1][1]), [0,0,255], 4)
cv2.line(image1, (src[3][0], src[3][1]), (src[0][0], src[0][1]), [0,0,255], 4) 
cv2.circle(image1, (src[2][0], src[2][1]), 2, [0,255,0], 12)
cv2.circle(image1, (src[3][0], src[3][1]), 2, [0,255,0], 12)

#right reference line original image
cv2.line(image1, (src[0][0], src[0][1]), (src[1][0], src[1][1]), [255,0,0], 4)
cv2.circle(image1, (src[0][0], src[0][1]), 2, [255,0,0], 12)
cv2.circle(image1, (src[1][0], src[1][1]), 2, [255,0,0], 12)

filename2 = 'output_images/straight_lines2_undist.jpg'
image2 = plt.imread(filename2)
src = getWarpSrcPts(image2) 
dst = getWarpDstPts(image2) 

warped2,M,invM = warper(image2, src, dst)

plt.imsave('output_images/straight_lines2_warped.jpg',warped2)
#left reference line warped image
cv2.line(warped2, (dst[2][0], dst[2][1]), (dst[3][0], dst[3][1]), [0,255,0], 4)
cv2.circle(warped2, (dst[2][0], dst[2][1]), 2, [0,255,0], 12)
cv2.circle(warped2, (dst[3][0], dst[3][1]), 2, [0,255,0], 12)

#right reference line warped image
cv2.line(warped2, (dst[0][0], dst[0][1]), (dst[1][0], dst[1][1]), [255,0,0], 4)
cv2.circle(warped2, (dst[0][0], dst[0][1]), 2, [255,0,0], 12)
cv2.circle(warped2, (dst[1][0], dst[1][1]), 2, [255,0,0], 12)

#left reference line original image
cv2.line(image2, (src[2][0], src[2][1]), (src[3][0], src[3][1]), [0,255,0], 4) 
cv2.line(image2, (src[2][0], src[2][1]), (src[1][0], src[1][1]), [0,0,255], 4)
cv2.line(image2, (src[3][0], src[3][1]), (src[0][0], src[0][1]), [0,0,255], 4) 
cv2.circle(image2, (src[2][0], src[2][1]), 2, [0,255,0], 12)
cv2.circle(image2, (src[3][0], src[3][1]), 2, [0,255,0], 12)

#right reference line original image
cv2.line(image2, (src[0][0], src[0][1]), (src[1][0], src[1][1]), [255,0,0], 4)
cv2.circle(image2, (src[0][0], src[0][1]), 2, [255,0,0], 12)
cv2.circle(image2, (src[1][0], src[1][1]), 2, [255,0,0], 12)


f, ((ax1,ax2),(ax3,ax4)) = plt.subplots(2, 2, figsize=(18, 7))
f.tight_layout()
ax1.imshow(image1)
ax1.set_title('Original image 1 with reference lines', fontsize=10)
ax2.imshow(warped1)
ax2.set_title('Img 1 Undistorted and warped', fontsize=10)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

ax3.imshow(image2)
ax3.set_title('Original image 2 with reference lines', fontsize=10)
ax4.imshow(warped2)
ax4.set_title('Img2 Undistorted and warped', fontsize=10)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

# Save the full figure...
f.savefig('output_images/warped_straight_lines.jpg')

## apply color, gradient, undistort and warp to straight lines


In [9]:



'''
filename1 = 'output_images/straight_lines1_undist.jpg'

image1 = mpimg.imread(filename1)
result1,result1BW = saturSobelRed(image1, s_thresh=(150, 255), sx_thresh=(20, 100),kernelS=5,gauss_blur=True,gauss_k_size=3)
singleCh1 = merge_color_channels(result1)
src = getWarpSrcPts(singleCh1) 
dst = getWarpDstPts(singleCh1) 
warped1,M,invM = warper(singleCh1, src, dst)


filename2 = 'output_images/straight_lines2_undist.jpg'
image2 = plt.imread(filename2)
result2,result2BW = saturSobelRed(image2, s_thresh=(150, 255), sx_thresh=(20, 100),kernelS=5,gauss_blur=True,gauss_k_size=3)
singleCh2 = merge_color_channels(result2)
src = getWarpSrcPts(singleCh2) 
dst = getWarpDstPts(singleCh2) 
warped2,M,invM = warper(singleCh2, src, dst)


f, ((ax1,ax2),(ax3,ax4)) = plt.subplots(2, 2, figsize=(18, 7))
f.tight_layout()
ax1.imshow(result1)
ax1.set_title('Img 1 thresholded', fontsize=10)
ax2.imshow(warped1,cmap='gray')
ax2.set_title('Img 1 binary warped after thresholds', fontsize=10)
ax3.imshow(result2)
ax3.set_title('Img2 thresholded', fontsize=10)
ax4.imshow(warped2,cmap='gray')
ax4.set_title('Img 2 binary warped after thresholds', fontsize=10)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

# Save the full figure...
f.savefig('output_images/thresholded_warped_straight_lines.jpg')
'''


"\nfilename1 = 'output_images/straight_lines1_undist.jpg'\n\nimage1 = mpimg.imread(filename1)\nresult1,result1BW = saturSobelRed(image1, s_thresh=(150, 255), sx_thresh=(20, 100),kernelS=5,gauss_blur=True,gauss_k_size=3)\nsingleCh1 = merge_color_channels(result1)\nsrc = getWarpSrcPts(singleCh1) \ndst = getWarpDstPts(singleCh1) \nwarped1,M,invM = warper(singleCh1, src, dst)\n\n\nfilename2 = 'output_images/straight_lines2_undist.jpg'\nimage2 = plt.imread(filename2)\nresult2,result2BW = saturSobelRed(image2, s_thresh=(150, 255), sx_thresh=(20, 100),kernelS=5,gauss_blur=True,gauss_k_size=3)\nsingleCh2 = merge_color_channels(result2)\nsrc = getWarpSrcPts(singleCh2) \ndst = getWarpDstPts(singleCh2) \nwarped2,M,invM = warper(singleCh2, src, dst)\n\n\nf, ((ax1,ax2),(ax3,ax4)) = plt.subplots(2, 2, figsize=(18, 7))\nf.tight_layout()\nax1.imshow(result1)\nax1.set_title('Img 1 thresholded', fontsize=10)\nax2.imshow(warped1,cmap='gray')\nax2.set_title('Img 1 binary warped after thresholds', fontsize

## apply color, gradient, undistort and warp to other test images (curved lines)

In [10]:


def binary_compare(bin_sobelx,sobelx_bin):
    w = sobelx_bin.shape[1]
    h = sobelx_bin.shape[0]
    for x in range(0,w-1):
        for y in range(0,h-1):
            if sobelx_bin[y,x] != bin_sobelx[y,x]:
                print("something wrong here: {}{}".format(x,y))

images = glob.glob('test_images/test*.jpg')    

for im in images:
    imNum = im.split('test_images/test')[1].split('.jpg')[0]
    image = mpimg.imread(im)

    warped,M,invM = undistort_and_warp(image)
    binary_or_img,bin_sobelx,bin_s_channel,bin_r_channel,bin_sobel_mag = colors_and_gradients(warped)

    
    #convert the image to three channel for vis purposes
    binary_or_img_color = cv2.cvtColor(binary_or_img, cv2.COLOR_GRAY2RGB) *255
    #left reference line warped image
    cv2.line(binary_or_img_color, (dst[2][0], dst[2][1]), (dst[3][0], dst[3][1]), [0,255,0], 4)
    #right reference line warped image
    cv2.line(binary_or_img_color, (dst[0][0], dst[0][1]), (dst[1][0], dst[1][1]), [255,0,0], 4)    
    
    f, ((ax1,ax2),(ax3,ax4),(ax5,ax6)) = plt.subplots(3, 2, figsize=(18, 7))
    f.tight_layout()
    ax1.imshow(warped)
    ax1.set_title('Warped test image {}'.format(imNum), fontsize=10)
    ax2.imshow(cv2.cvtColor(bin_s_channel, cv2.COLOR_GRAY2RGB) *255)
    ax2.set_title('Binary saturation channel', fontsize=10)
    ax3.imshow(cv2.cvtColor(bin_r_channel, cv2.COLOR_GRAY2RGB) *255)
    ax3.set_title('Binary red channel', fontsize=10)
    ax4.imshow(cv2.cvtColor(bin_sobelx, cv2.COLOR_GRAY2RGB) *255)
    ax4.set_title('Binary sobelx channel', fontsize=10)
    ax5.imshow(cv2.cvtColor(bin_sobel_mag, cv2.COLOR_GRAY2RGB) *255)
    ax5.set_title('Binary sobel magnitude (x and y) channel', fontsize=10)
    ax6.imshow(binary_or_img_color)
    ax6.set_title('Binary OR of all channels above', fontsize=10)    
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

## Finding lane pixels with sliding windows and polynoms

In [11]:
def build_poly_lines(img_shape, left_fit,right_fit):
    # Generate x and y values for plotting
    ploty = np.linspace(0, img_shape[0]-1, img_shape[0])
    
    try:           
        right_fitx = ploty**2*right_fit[0] + ploty*right_fit[1] + right_fit[2]
        left_fitx = ploty**2*left_fit[0] + ploty*left_fit[1] + left_fit[2]
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty


    return left_fitx, right_fitx, ploty

def fit_poly(leftx, lefty, rightx, righty):
    ### TO-DO: Fit a second order polynomial to each with np.polyfit() ###
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    return left_fit, right_fit


def lane_search_with_windows(binary_warped):
    # Find our lane pixels first
    leftx, lefty, rightx, righty, out_img = find_lane_pixels(binary_warped)

    ### TO-DO: Fit a second order polynomial to each using `np.polyfit` ###
    left_fit,right_fit = fit_poly(leftx, lefty, rightx, righty)

    # Generate x and y values for plotting
    imsize = (binary_warped.shape[0]-1, binary_warped.shape[0])
    left_fitx, right_fitx, ploty =  build_poly_lines(binary_warped.shape, left_fit,right_fit)

    ## Visualization ##
    # Colors in the left and right lane regions
    out_img[lefty, leftx] = [255, 0, 0]
    out_img[righty, rightx] = [0, 0, 255]

    # Plots the left and right polynomials on the lane lines
    #plt.plot(left_fitx, ploty, color='yellow')
    #plt.plot(right_fitx, ploty, color='yellow')

    return out_img,left_fitx, right_fitx, ploty,left_fit, right_fit

     
def lane_search_around_poly(binary_warped,left_fit_prev,right_fit_prev):
    # HYPERPARAMETER
    # Choose the width of the margin around the previous polynomial to search
    # The quiz grader expects 100 here, but feel free to tune on your own!
    margin = 100

    # Grab activated pixels
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    ### TO-DO: Set the area of search based on activated x-values ###
    ### within the +/- margin of our polynomial function ###
    ### Hint: consider the window areas for the similarly named variables ###
    ### in the previous quiz, but change the windows to our new search area ###
    
    left_lane_inds = ((nonzerox <= ((nonzeroy**2)*left_fit_prev[0]+nonzeroy*left_fit_prev[1]+left_fit_prev[2])+margin)&
                      (nonzerox > ((nonzeroy**2)*left_fit_prev[0]+nonzeroy*left_fit_prev[1]+left_fit_prev[2])-margin))
    right_lane_inds = ((nonzerox <= ((nonzeroy**2)*right_fit_prev[0]+nonzeroy*right_fit_prev[1]+right_fit_prev[2])+margin)&
                      (nonzerox > ((nonzeroy**2)*right_fit_prev[0]+nonzeroy*right_fit_prev[1]+right_fit_prev[2])-margin))
    
    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]


    # Fit new polynomials
    left_fit, right_fit = fit_poly(leftx, lefty, rightx, righty)

    #build lines
    left_fitx, right_fitx, ploty = build_poly_lines(binary_warped.shape, left_fit,right_fit)

    #left_fitx, right_fitx, ploty = fit_poly_old(binary_warped.shape, leftx, lefty, rightx, righty)
    
    ## Visualization ##
    # Create an image to draw on and an image to show the selection window
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    window_img = np.zeros_like(out_img)
    
    # Color in left and right line pixels
    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

    # Generate a polygon to illustrate the search window area
    # And recast the x and y points into usable format for cv2.fillPoly()
    left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
    left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, 
                              ploty])))])
    left_line_pts = np.hstack((left_line_window1, left_line_window2))
    
    right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
    right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, 
                              ploty])))])
    right_line_pts = np.hstack((right_line_window1, right_line_window2))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
    cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
    result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)
    
    # Plot the polynomial lines onto the image
    plt.plot(left_fitx, ploty, color='yellow')
    plt.plot(right_fitx, ploty, color='yellow')
    ## End visualization steps ##
    
    return result,left_fitx, right_fitx, ploty,left_fit, right_fit


def measure_curvature_real(ym_per_pix,xm_per_pix,ploty,left_fit_cr,right_fit_cr):
    '''
    Calculates the curvature of polynomial functions in meters.
    '''    
    # Define y-value where we want radius of curvature
    # We'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = np.max(ploty)
    
    ##### TO-DO: Implement the calculation of R_curve (radius of curvature) #####
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    
    return left_curverad, right_curverad

def find_lane_pixels(binary_warped):
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint
    #327 and 971 are starting points
    
    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = 9
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        
        ### TO-DO: Find the four below boundaries of the window ###
        win_xleft_low = leftx_current - margin  
        win_xleft_high = leftx_current + margin  
        
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),
        (win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),
        (win_xright_high,win_y_high),(0,255,0), 2) 
        
        ### TO-DO: Identify the nonzero pixels in x and y within the window ###
        good_left_inds = ((nonzerox >= win_xleft_low) & (nonzerox <win_xleft_high) &
        (nonzeroy>=win_y_low) & (nonzeroy <win_y_high)).nonzero()[0]
        good_right_inds = ((nonzerox >=win_xright_low)&(nonzerox <win_xright_high)&
                           (nonzeroy>=win_y_low) & (nonzeroy <win_y_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        ### TO-DO: If you found > minpix pixels, recenter next window ###
        ### (`right` or `leftx_current`) on their mean position ###
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty, out_img

def car_lanes_offset(left,right,imwidth):
    imMid = imwidth//2
    laneMid = left+(right-left)//2
    return laneMid-imMid

def fill_driving_space(grayImg,left_fitx, right_fitx, ploty):

    h, w = grayImg.shape[:2]

    pointsL = np.array([[[xi, yi]] for xi, yi in zip(left_fitx, ploty) if (0<=xi<w and 0<=yi<h)]).astype(np.int32)
    pointsR = np.array([[[xi, yi]] for xi, yi in zip(right_fitx, ploty) if (0<=xi<w and 0<=yi<h)]).astype(np.int32)
    pointsR = np.flipud(pointsR)
    points = np.concatenate((pointsL, pointsR))

    driving_space = grayImg.copy()
    driving_space = cv2.cvtColor(driving_space,cv2.COLOR_GRAY2RGB)
    #color driving space
    cv2.fillPoly(driving_space, [points], color=[0,255,0])
    #add left line overlay
    cv2.polylines(driving_space, [pointsL], color=[255,0,0], isClosed = True,thickness = 20)
    #add right line overlay
    cv2.polylines(driving_space, [pointsR], color=[255,0,0], isClosed = True,thickness = 20)   
    
    return driving_space

# Polynomial fit values from the previous frame
# Make sure to grab the actual values from the previous step in your project!
#left_fit_prev = np.array([ 2.13935315e-04, -3.77507980e-01,  4.76902175e+02])
#right_fit_prev = np.array([4.17622148e-04, -4.93848953e-01,  1.11806170e+03])

# Define conversions in x and y from pixels space to meters
ym_per_pix = 30/720 # meters per pixel in y dimension
xm_per_pix = 3.7/700 # meters per pixel in x dimension












## testing pipeline on warped test images

In [12]:
images = glob.glob('test_images/test*.jpg')    

for im in images:
    imNum = im.split('test_images/test')[1].split('.jpg')[0]

    image = plt.imread(im)

    warped,M,invM = undistort_and_warp(image)
    binary_or_img,bin_sobelx,bin_s_channel,bin_r_channel,bin_sobel_mag = colors_and_gradients(warped)
    

    #result,left_fitx, right_fitx, ploty,left_fit, right_fit = lane_search_around_poly(binary_warped,left_fit_prev,right_fit_prev))
    lineImg1,left_fitx, right_fitx, ploty,left_fit, right_fit = lane_search_with_windows(binary_or_img)
    
    
    #print(left_fit.shape,left_fit_cr.shape)
    # Calculate the radius of curvature in meters for both lane lines
    leftcurv, rightcurv = measure_curvature_real(ym_per_pix,xm_per_pix,ploty,left_fit,right_fit)
    car_offset = car_lanes_offset(left_fitx[-1],right_fitx[-1],lineImg1.shape[1])
    s = "Left curv:{:.2f}[m] \nRight curv:{:.2f}[m]\nCar offset:{:.2f}[m]".format(leftcurv,rightcurv,car_offset*xm_per_pix)
    x = 520
    y = 100
    
   
    f, ((ax1,ax2)) = plt.subplots(1, 2, figsize=(18, 7))
    f.tight_layout()
    ax1.imshow(warped)
    cv2.imwrite("test%s.jpg" % imNum, warped)
    ax1.set_title('Warped result test image {}'.format(imNum),  fontsize=10)
    plt.plot(left_fitx, ploty, color='yellow')
    plt.plot(right_fitx, ploty, color='yellow')
    ax2.imshow(lineImg1)
    ax2.text(x, y, s, fontsize=10, color='white')
    ax2.set_title('Detected lanes with\n corresponding polynom drawn', fontsize=10)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
    

/Users/giorgio/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


In [13]:

def warped_binary(image):
    
    warped,M,invM = undistort_and_warp(image)
    binary_or_img,bin_sobelx,bin_s_channel,bin_r_channel,bin_sobel_mag = colors_and_gradients(warped)
    
    return warped,M,invM,binary_or_img


def add_curv_offset_text(merged,leftcurv,rightcurv):
    
    car_offset = car_lanes_offset(left_fitx[-1],right_fitx[-1],image.shape[1])

    leftTxt = "Left curv:{:.2f}[m]".format(leftcurv)    
    rigtTxt = "Right curv:{:.2f}[m]".format(rightcurv)
    offsetTxt = "Car offset:{:.2f}[m]".format(car_offset*xm_per_pix)

    position = (400,100)
    cv2.putText(
         merged, #numpy array on which text is written
         leftTxt, #text
         position, #position at which writing has to start
         cv2.FONT_HERSHEY_SIMPLEX, #font family
         1, #font size
         (255, 255, 255, 255), #font color
         3) #font stroke 
    position = (400,140)
    cv2.putText(
         merged, #numpy array on which text is written
         rigtTxt, #text
         position, #position at which writing has to start
         cv2.FONT_HERSHEY_SIMPLEX, #font family
         1, #font size
         (255, 255, 255, 255), #font color
         3) #font stroke 
    position = (400,180)
    cv2.putText(
         merged, #numpy array on which text is written
         offsetTxt, #text
         position, #position at which writing has to start
         cv2.FONT_HERSHEY_SIMPLEX, #font family
         1, #font size
         (255, 255, 255, 255), #font color
         3) #font stroke   



def process_frame(image):
    global LinesRingBuffer
    
    warped,M,invM,binary_warped = warped_binary(image)

    valid_l_fit_x = None
    valid_l_cr = None
    valid_r_fit_x = None
    valid_r_cr = None
    
    if LinesRingBuffer.isFull():
        print("enough lines for averaging and searching")
        #get the last poly values (averaged across the buffer)
        
        lines = LinesRingBuffer.data
        left_avg_coeffs,right_avg_coeffs = averaging_poly(lines) 
        #left_fitx, right_fitx, ploty = build_poly_lines(binary_warped.shape,left_avg_coeffs,right_avg_coeffs) 
        
        #use the poly coeffs from last line to search new lines
        result,left_fitx, right_fitx, ploty,left_fit, right_fit = lane_search_around_poly(binary_warped,left_avg_coeffs,right_avg_coeffs)
        
        #if new line is detected within the margin, append it to the line buffer and use it
        
        #else empty the buffer, search line using sliding windows and append it
        
        result,valid_l_fit_x, valid_r_fit_x, ploty,left_fit, right_fit = lane_search_with_windows(binary_warped)
        valid_l_cr, valid_r_cr = measure_curvature_real(ym_per_pix,xm_per_pix,ploty,left_fit,right_fit)
        left=Line()
        right=Line()
        left.setLatestFitx(valid_l_fit_x)
        left.setLatestFit(left_fit)
        left.setCurvature(valid_l_cr)
        left.setDetectedFlag(True)
        right.setLatestFitx(valid_r_fit_x)
        right.setLatestFit(right_fit)
        right.setCurvature(valid_r_cr)
        right.setDetectedFlag(True)

        LinesRingBuffer.append((left,right))
        

            
            
    else:
        #fill the buffer with lines detected with the sliding windows method
        result,valid_l_fit_x, valid_r_fit_x, ploty,left_fit, right_fit = lane_search_with_windows(binary_warped)
        valid_l_cr, valid_r_cr = measure_curvature_real(ym_per_pix,xm_per_pix,ploty,left_fit,right_fit)
        left=Line()
        right=Line()
        left.setLatestFitx(valid_l_fit_x)
        left.setLatestFit(left_fit)
        left.setCurvature(valid_l_cr)
        left.setDetectedFlag(True)
        right.setLatestFitx(valid_r_fit_x)
        right.setLatestFit(right_fit)
        right.setCurvature(valid_r_cr)
        right.setDetectedFlag(True)

        LinesRingBuffer.append((left,right)) 

 
        
    
    #lineImg1,left_fitx, right_fitx, ploty,left_fit, right_fit = lane_search_with_windows(binary)
    #lineImg1,left_fitx, right_fitx, ploty,left_fit, right_fit = lane_search_with_windows(binary)
       
    
    warpedGray = cv2.cvtColor(warped,cv2.COLOR_RGB2GRAY)
    
    driving_space = fill_driving_space(warpedGray,valid_l_fit_x, right_fitx, ploty)
    img_size = (driving_space.shape[1], driving_space.shape[0])
    unwarped = cv2.warpPerspective(driving_space,invM,img_size,flags=cv2.INTER_LINEAR)    
    merged = cv2.addWeighted(image, 1, unwarped, 0.3, 0)
    add_curv_offset_text(merged,valid_l_cr,valid_r_cr)
    return merged


class RingBuffer:
    """ class that implements a not-yet-full buffer 
    source: https://www.oreilly.com/library/view/python-cookbook/0596001673/ch05s19.html"""
    
    def __init__(self,size_max):
        self.max = size_max
        self.data = []
        self.full = False
        
    class __Full:
        """ class that implements a full buffer """
        def append(self, x):
            """ Append an element overwriting the oldest one. """
            self.data[self.cur] = x
            self.cur = (self.cur+1) % self.max
        
        def isFull(self):
            return self.full    
        
        def get(self):
            """ return list of elements in correct order """
            return self.data[self.cur:]+self.data[:self.cur]

    def append(self,x):
        """append an element at the end of the buffer"""
        self.data.append(x)
        if len(self.data) == self.max:
            self.full = True
            self.cur = 0
            
            # Permanently change self's class from non-full to full
            self.__class__ = self.__Full
    
    def isFull(self):
        return self.full

    def get(self):
        """ Return a list of elements from the oldest to the newest. """
        return self.data
 

       

In [14]:
 def averaging_poly(lines):
    
    left_coeffs = []
    avg_left = []
    right_coeffs = []
    avg_right = []
    
    for l,r in lines:
        left_coeffs.append(l.getLatestFit())
        right_coeffs.append(r.getLatestFit())

    avg_left = np.mean(np.stack([left_coeffs]), axis=0)
    avg_right = np.mean(np.stack([right_coeffs]), axis=0)

    return avg_left, avg_right


In [15]:
class Line():
    def __init__(self):
        # was the line detected in the last iteration?
        self.detected = False  
        # x values of the last n fits of the line
        self.recent_xfitted = [] 
        #average x values of the fitted line over the last n iterations
        self.bestx = None     
        #polynomial coefficients averaged over the last n iterations
        self.best_fit = None  
        #polynomial coefficients for the most recent fit
        self.current_fit = [np.array([False])]  
        #radius of curvature of the line in some units
        self.radius_of_curvature = None 
        #distance in meters of vehicle center from the line
        self.line_base_pos = None 
        #difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float') 

        #x values for detected line pixels
        self.allx = None  
        #y values for detected line pixels
        self.ally = None
    
    def setDetectedFlag(self, flag):
        self.detected = flag
        
    def getDetectedFlag(self):
        return self.detected
        
    def setLatestFitx(self,fitx):
        self.recent_xfitted = fitx
        
    def getLatestFitx(self):
        return self.recent_xfitted 
    
    def setLatestFit(self,fit_coeffs):
        self.current_fit = fit_coeffs
        
    def getLatestFit(self):
        return self.current_fit     

    def setCurvature(self,curv):
        self.radius_of_curvature = curv
        
    def getCurvature(self):
        return self.radius_of_curvature  
    
 


## process video

In [17]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML

LinesRingBuffer = RingBuffer(5)
output='output.mp4'
clip1 = VideoFileClip('project_video.mp4')
#clip1 = VideoFileClip('challenge_video.mp4')

clip = clip1.fl_image(process_frame).subclip(0,.5)
%time clip.write_videofile(output, audio=False)

UnboundLocalError: local variable 'right_fitx' referenced before assignment

In [ ]:
from IPython.display import Video

#Video('project_video.mp4')

from IPython.display import HTML

HTML("""
<video width="960" height="540" controls>
  <source src='output.mp4'>
</video>
""")